# End-to-End Demonstration: Canvas

*Goal*: run the Canvas Extractor and then upload the results into an `LMS` database.

## Software Requirements

1. Be sure to install Python 3.9; if you have multiple versions, make sure that the `python` command runs version 3.9.x. You can confirm your version by running `python --version` at a command prompt.
1. Microsoft SQL Server 2017 or 2019, in Windows or Linux.

## Getting Started

1. Confirm you have [poetry](https://python-poetry.org) installed (`poetry --version`).
1. Follow the [notebook instructions](../../src/canvas-extractor/README.md) to install dependencies used by this notebook.
1. Create an `LMS` database in SQL Server.
1. Please read through the notebook and fill in the two text boxes below before executing the entire notebook.

In [8]:
# Load some utilities
from IPython.display import display, Markdown
import ipywidgets as widgets

# Setup logging
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# This logging produces too much information in the underlying Canvas API package, 
# leading to personally-identifiable information in the logs. The following configuration
# reduces that part of the logging configuration back to the WARNING level.
canvasapi_logger = logging.getLogger("canvasapi")
if canvasapi_logger.getEffectiveLevel() < logging.WARN:
    canvasapi_logger.setLevel(logging.WARN)

## Prepare Input Data

Update the variables in the next block as needed.

In [2]:
LOG_LEVEL = "INFO"
OUTPUT_DIRECTORY = "c-data"
SYNC_DATABASE_DIRECTORY=OUTPUT_DIRECTORY
START_DATE="2021-02-01"
END_DATE="2021-07-28"
DB_ENGINE = "mssql"
DB_SERVER = "localhost"
DB_NAME = "LMS"
DB_PORT = 1433
EXTRACT_ASSIGNMENTS = True
EXTRACT_ACTIVITIES = False
EXTRACT_ATTENDANCE = False
EXTRACT_GRADES = False

Run the next cell to create text input boxes for providing the agency's Canvas URL and a Canvas access token. [Instructions for acquiring an access token](https://community.canvaslms.com/t5/Student-Guide/How-do-I-manage-API-access-tokens-as-a-student/ta-p/273). Then run the next cell down to capture the values before running the last cells.

In [3]:
def get_access_token(token, base_url):
    return token, base_url

token_widget = widgets.Text(
    description="Access Token"
)
base_url_widget = widgets.Text(
    description="Base URL"
)
result = widgets.interactive(get_access_token, token=token_widget, base_url=base_url_widget)
display(result)

interactive(children=(Text(value='', description='Access Token'), Text(value='', description='Base URL'), Outp…

In [10]:
TOKEN = result.children[0].value
CANVAS_BASE_URL = result.children[1].value
print("Captured token and base URL.")

Captured token and base URL.


## Run the Canvas Extractor

In [11]:
from edfi_canvas_extractor.helpers.arg_parser import MainArguments as c_args
from edfi_canvas_extractor import extract_facade

arguments = c_args(
    base_url=CANVAS_BASE_URL,
    access_token=TOKEN,
    output_directory=OUTPUT_DIRECTORY,
    log_level=LOG_LEVEL,
    start_date=START_DATE,
    end_date=END_DATE,
    sync_database_directory=SYNC_DATABASE_DIRECTORY,
    extract_assignments=EXTRACT_ASSIGNMENTS,
    extract_activities=EXTRACT_ACTIVITIES,
    extract_attendance=EXTRACT_ATTENDANCE,
    extract_grades=EXTRACT_GRADES,
)

extract_facade.run(arguments)

INFO:edfi_canvas_extractor.extract_facade:Starting Ed-Fi LMS Canvas Extractor
INFO:edfi_canvas_extractor.extract_facade:Extracting Courses from Canvas API
INFO:edfi_canvas_extractor.api.courses:Pulling course data
INFO:edfi_canvas_extractor.extract_facade:Extracting Sections from Canvas API
INFO:edfi_canvas_extractor.api.sections:Pulling section data
INFO:edfi_canvas_extractor.extract_facade:Writing LMS UDM Sections to CSV file
INFO:edfi_lms_extractor_lib.csv_generation.write:Generated file => c-data\sections\2021-09-03-16-25-20.csv
INFO:edfi_canvas_extractor.extract_facade:Extracting Students from Canvas API
INFO:edfi_canvas_extractor.api.students:Pulling student data
INFO:edfi_canvas_extractor.extract_facade:Writing LMS UDM Users to CSV file
INFO:edfi_lms_extractor_lib.csv_generation.write:Generated file => c-data\users\2021-09-03-16-25-46.csv
INFO:edfi_canvas_extractor.extract_facade:Extracting Enrollments from Canvas API
INFO:edfi_canvas_extractor.client_facade:There are no active 

INFO:edfi_lms_extractor_lib.csv_generation.write:Generated file => c-data\section=616\section-associations\2021-09-03-16-26-07.csv
INFO:edfi_lms_extractor_lib.csv_generation.write:Generated file => c-data\section=619\section-associations\2021-09-03-16-26-07.csv
INFO:edfi_lms_extractor_lib.csv_generation.write:Generated file => c-data\section=642\section-associations\2021-09-03-16-26-07.csv
INFO:edfi_lms_extractor_lib.csv_generation.write:Generated file => c-data\section=751\section-associations\2021-09-03-16-26-07.csv
INFO:edfi_canvas_extractor.extract_facade:Extracting Assignments from Canvas API
INFO:edfi_canvas_extractor.api.assignments:Pulling assignment data
INFO:edfi_canvas_extractor.extract_facade:Writing LMS UDM Assignments to CSV files
INFO:edfi_lms_extractor_lib.csv_generation.write:Generated file => c-data\section=103\assignments\2021-09-03-16-26-40.csv
INFO:edfi_lms_extractor_lib.csv_generation.write:Generated file => c-data\section=104\assignments\2021-09-03-16-26-40.csv
I

INFO:edfi_canvas_extractor.client_facade:Skipping submissions for section id 609 - No data returned by API
INFO:edfi_canvas_extractor.client_facade:Skipping submissions for section id 612 - No data returned by API
INFO:edfi_canvas_extractor.client_facade:Skipping submissions for section id 614 - No data returned by API
INFO:edfi_canvas_extractor.client_facade:Skipping submissions for section id 613 - No data returned by API
INFO:edfi_canvas_extractor.client_facade:Skipping submissions for section id 617 - No data returned by API
INFO:edfi_canvas_extractor.client_facade:Skipping submissions for section id 615 - No data returned by API
INFO:edfi_canvas_extractor.client_facade:Skipping submissions for section id 616 - No data returned by API
INFO:edfi_canvas_extractor.client_facade:Skipping submissions for section id 619 - No data returned by API
INFO:edfi_canvas_extractor.client_facade:Skipping submissions for section id 618 - No data returned by API
INFO:edfi_canvas_extractor.client_fac

## Run the Learning Management System Data Store Loader (LMS-DS-Loader)

The default setup below uses Windows integrated security. For username/password security, please review the commented-out code.

In [3]:
from edfi_lms_ds_loader.helpers.argparser import MainArguments as lms_args
from edfi_lms_ds_loader import loader_facade

arguments = lms_args(
    OUTPUT_DIRECTORY,
    DB_ENGINE,
    LOG_LEVEL,
    DB_SERVER,
    DB_NAME,
    DB_PORT,
)
arguments.build_mssql_adapter_with_integrated_security()

# For password auth, comment out the lines above and uncomment the next ones:
# arguments = lms_args(
#     OUTPUT_DIRECTORY,
#     DB_ENGINE,
#     LOG_LEVEL,
#     DB_SERVER,
#     DB_PORT,
#     DB_NAME,
# )
# arguments.build_mssql_adapter(your_db_user, your_db_password)


loader_facade.run_loader(arguments)